In [1]:
options(warn = -1)

In [2]:
library(lubridate)
library(rgdal)


Attaching package: 'lubridate'


The following objects are masked from 'package:base':

    date, intersect, setdiff, union


Loading required package: sp

rgdal: version: 1.5-23, (SVN revision 1121)
Geospatial Data Abstraction Library extensions to R successfully loaded
Loaded GDAL runtime: GDAL 3.2.1, released 2020/12/29
Path to GDAL shared files: C:/Users/clid1852/Documents/R/win-library/4.0/rgdal/gdal
GDAL binary built with GEOS: TRUE 
Loaded PROJ runtime: Rel. 7.2.1, January 1st, 2021, [PJ_VERSION: 721]
Path to PROJ shared files: C:/Users/clid1852/Documents/R/win-library/4.0/rgdal/proj
PROJ CDN enabled: TRUE
Linking to sp version:1.4-5
To mute warnings of possible GDAL/OSR exportToProj4() degradation,
use options("rgdal_show_exportToProj4_warnings"="none") before loading rgdal.
Overwritten PROJ_LIB was C:/Users/clid1852/Documents/R/win-library/4.0/rgdal/proj



In [3]:
data <- read.csv('T:/Tableau/tableauBikesOnBuses/Datasources/MonthlyBikesOnBuses.csv')

In [4]:
names(data)

[1] "stop"      "srv"       "date"      "block"     "trip_end"  "time"     
 [7] "route"     "dir"       "stop_name" "bus"       "odometer"  "desc"     
[13] "qty"       "MonthYear" "longitude" "latitude"

In [5]:
head(data)

,stop,srv,date,block,trip_end,time,route,dir,stop_name,bus,odometer,desc,qty,MonthYear,longitude,latitude
,<int>,<chr>,<chr>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<dbl>,<chr>,<int>,<chr>,<dbl>,<dbl>
1,1,wkd,2013-01-18 00:00:00,67,1899-12-31 18:21:00,1899-12-31 18:10:00,11,O,E/S of 58th N of Main,1002,98.11,bike on rack,1,January 2013,-122.9267,44.04634
2,1,wkd,2013-01-10 00:00:00,2,1899-12-31 09:56:00,1899-12-31 09:44:00,11,O,E/S of 58th N of Main,1114,71.08,bike on rack,1,January 2013,-122.9267,44.04634
3,1,wkd,2013-01-10 00:00:00,60,1899-12-31 16:54:00,1899-12-31 16:45:00,11,O,E/S of 58th N of Main,7102,119.29,bike on rack,1,January 2013,-122.9267,44.04634
4,1,wkd,2013-01-29 00:00:00,60,1899-12-31 17:52:00,1899-12-31 17:39:00,11,O,E/S of 58th N of Main,7102,132.76,bike on rack,1,January 2013,-122.9267,44.04634
5,1,wkd,2013-01-09 00:00:00,25,1899-12-31 15:35:00,1899-12-31 15:20:00,11,O,E/S of 58th N of Main,1004,126.65,bike on rack,1,January 2013,-122.9267,44.04634
6,2,wkd,2013-01-30 00:00:00,48,1899-12-31 08:27:00,1899-12-31 08:13:00,11,O,E/S of 58th S of D,6218,5.11,bike on rack,1,January 2013,-122.9262,44.04946


In [6]:
data$date <- as.Date(data$date, "%Y-%m-%d")

In [7]:
data$year <- year(data$date)

In [8]:
dat <- data[data$year == 2021,]

In [9]:
dim(dat)

[1] 49945    17

In [10]:
dat$weekday <- as.character(wday(dat$date, label=TRUE, abbr=FALSE))

In [11]:
dat$month <- months(dat$date)

In [12]:
dat$season <- ifelse(dat$month %in% c("December", "January", "February"), "Winter",
                    ifelse(dat$month %in% c("March", "April", "May"), "Spring",
                          ifelse(dat$month %in% c("June", "July", "August"), "Summer", "Fall")))

In [13]:
dat$seasonOrder <- ifelse(dat$season == "Spring", 1, 
                              ifelse(dat$season == "Summer", 2, 
                                     ifelse(dat$season == "Fall", 3, 4)))

In [14]:
dat$monthOrder <- ifelse(dat$month == "January", 1, 
                           ifelse(dat$month == "February", 2, 
                               ifelse(dat$month == "March", 3, 
                                   ifelse(dat$month == "April", 4, 
                                       ifelse(dat$month == "May", 5, 
                                           ifelse(dat$month == "June", 6, 
                                               ifelse(dat$month == "July", 7, 
                                                   ifelse(dat$month == "August", 8, 
                                                       ifelse(dat$month == "September", 9, 
                                                           ifelse(dat$month == "October", 10, 
                                                               ifelse(dat$month == "November", 11, 12)))))))))))

In [15]:
dat$weekdayOrder <- ifelse(dat$weekday == "Monday", 1, 
                              ifelse(dat$weekday == "Tuesday", 2, 
                                     ifelse(dat$weekday == "Wednesday", 3, 
                                     ifelse(dat$weekday == "Thursday", 4, 
                                            ifelse(dat$weekday == "Friday", 5, 
                                                ifelse(dat$weekday == "Saturday", 6, 7))))))

In [16]:
datedf <- unique(dat[,c('date', 'season', 'month', 'weekday', 'seasonOrder', 'monthOrder', 'weekdayOrder')])

In [17]:
stop.path <- "T:/Data/LTD Data/StopsSince2011"

In [18]:
stops <- readOGR(dsn = stop.path, layer = "April 2020", verbose = FALSE, 
                   stringsAsFactors = FALSE)

In [19]:
names(stops)

[1] "stop_numbe" "stop_name"  "latitude"   "longitude"  "AvgBoards"

In [20]:
dim(stops@data)[1] == length(unique(stops$stop_numbe))

[1] TRUE

In [21]:
head(data$stop)

[1] 1 1 1 1 1 2

In [22]:
head(stops$stop_numbe)

[1] "00001" "00002" "00004" "00006" "00007" "00008"

In [23]:
zeros <- c("0", "00", "000", "0000")

In [24]:
dat$stop <- ifelse(nchar(dat$stop) == 5, dat$stop,
                    paste0(zeros[(5 - nchar(dat$stop))], dat$stop))

In [25]:
stopdf <- unique(dat[,c('stop', 'route')])

In [26]:
names(stops)[1] <- "stop"

In [27]:
stopdf <- merge(stopdf, stops@data[, -dim(stops)[2]], by="stop")

In [28]:
aggdata <- aggregate(x=list(qty = dat$qty), 
                     by=list(date = dat$date, stop = dat$stop), 
                     FUN=sum)

In [36]:
head(aggdata)

,date,stop,qty,out_qty,in_qty,route,stop_name,latitude,longitude,season,month,weekday,seasonOrder,monthOrder,weekdayOrder
,<date>,<chr>,<int>,<int>,<int>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>
1,2021-01-07,01897,2,1,1,41,S/S of 11th W of Commerce (Walmart),44.04799,-123.1701,Winter,January,Thursday,4,1,4
2,2021-01-07,01897,2,1,1,36,S/S of 11th W of Commerce (Walmart),44.04799,-123.1701,Winter,January,Thursday,4,1,4
3,2021-01-07,01897,2,1,1,93,S/S of 11th W of Commerce (Walmart),44.04799,-123.1701,Winter,January,Thursday,4,1,4
4,2021-01-08,01897,3,2,1,36,S/S of 11th W of Commerce (Walmart),44.04799,-123.1701,Winter,January,Friday,4,1,5
5,2021-01-08,01897,3,2,1,93,S/S of 11th W of Commerce (Walmart),44.04799,-123.1701,Winter,January,Friday,4,1,5
6,2021-01-08,01897,3,2,1,41,S/S of 11th W of Commerce (Walmart),44.04799,-123.1701,Winter,January,Friday,4,1,5


In [30]:
get_aggdata <- function(df=dat, dir="O"){
    df <- df[df$dir == dir,]
    aggdata <- aggregate(x=list(qty = df$qty), 
                     by=list(date = df$date, stop = df$stop), 
                     FUN=sum)

    if(dir=="O"){
        names(aggdata)[3] <- "out_qty"
    }else{
        names(aggdata)[3] <- "in_qty"
    }
    
    return(aggdata)    
}

In [31]:
aggdata <- merge(aggdata, get_aggdata(df=dat, dir="O"), by=c("stop", "date"))

In [32]:
aggdata <- merge(aggdata, get_aggdata(df=dat, dir="I"), by=c("stop", "date"))

In [34]:
aggdata <- merge(aggdata, stopdf, by="stop")

In [35]:
aggdata <- merge(aggdata, datedf, by="date")

In [37]:
dim(aggdata)

[1] 208  15

In [38]:
outpath <- "T:/DCProjects/StoryMap/BikeCounting/BikeOnBuses/Output"

In [39]:
write.csv(aggdata, paste0(outpath, "/Daily_Bikes_On_Buses.csv"), row.names = FALSE)

In [40]:
MPOBound <- readOGR(dsn = "V:/Data/Transportation", layer="MPO_Bound")

OGR data source with driver: ESRI Shapefile 
Source: "V:\Data\Transportation", layer: "MPO_Bound"
with 1 features
It has 3 fields


In [41]:
df2spdf <- function(df, lon_col_name, lat_col_name, trans = TRUE){
  lonlat <- sp::CRS("+proj=longlat +datum=WGS84 +ellps=WGS84 +towgs84=0,0,0")
  lon_col_no <- which(names(df)==lon_col_name)
  lat_col_no <- which(names(df)==lat_col_name)
  xy <- data.frame(df[,c(lon_col_no,lat_col_no)])
  coordinates(xy) <- c(lon_col_name, lat_col_name)
  proj4string(xy) <- lonlat
  spdf <- sp::SpatialPointsDataFrame(coords = xy, data = df)
  if(trans){
    spdf <- spTransform(spdf, CRS(proj4string(MPOBound)))
  }
  return(spdf)
}

In [42]:
spdf <- df2spdf(aggdata, 'longitude', 'latitude')

In [43]:
writeOGR(spdf, dsn=outpath, layer="Daily_Bikes_On_Buses", driver="ESRI Shapefile", overwrite_layer=TRUE)

In [44]:
sumdf <- aggregate(x=list(qty = aggdata$qty, 
                          out_qty = aggdata$out_qty, 
                          in_qty = aggdata$in_qty), 
                   by=list(stop = aggdata$stop), 
                   FUN=sum)

In [47]:
avgdf <- aggregate(x=list(avgqty = aggdata$qty), 
                   by=list(stop = aggdata$stop), 
                   FUN=mean)

In [48]:
sumdf$pct_out <- (sumdf$out_qty / sumdf$qty) * 100

In [49]:
sum_avg_df <- merge(sumdf, avgdf, by="stop")

In [50]:
sum_avg_df <- merge(sum_avg_df, stopdf, by="stop")

In [51]:
write.csv(sum_avg_df, paste0(outpath, "/Sum_Bikes_On_Buses.csv"), row.names = FALSE)

In [52]:
sum_avg_spdf <- df2spdf(sum_avg_df, 'longitude', 'latitude')

In [53]:
names(sum_avg_spdf)

[1] "stop"      "qty"       "out_qty"   "in_qty"    "pct_out"   "avgqty"   
 [7] "route"     "stop_name" "latitude"  "longitude"

In [54]:
writeOGR(sum_avg_spdf, dsn=outpath, layer="Sum_Bikes_On_Buses", driver="ESRI Shapefile", overwrite_layer=TRUE)